In [25]:
pip install playsound


Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install librosa==0.9.2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import librosa
import soundfile
import os, glob, pickle , sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import time
import tkinter as tk
from tkinter import ttk
from tkinter.filedialog import askopenfile 
import tkinter.messagebox
from playsound import playsound
import matplotlib.pyplot as plt
import wave, sys


A:\anaconda\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
filename = None
bgcolor = 'lightgreen'
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
root = tk.Tk()
root.title("Emotion Recognition from Speech")
canvas = tk.Canvas(root, width = 800, height = 500,  relief = 'raised' , bg=bgcolor)
canvas.pack()

labelHeading = tk.Label(root, text="Emotion Recognition from Speech",  fg="blue" , bg=bgcolor)
labelHeading.config(font=('helvetica', 26))
canvas.create_window(400, 50, window=labelHeading) 

labelSelectInput = tk.Label(root, text="Select Input:",  fg="black" , bg=bgcolor)
labelHeading.config(font=('helvetica', 16 , 'bold'))
canvas.create_window(100, 150, window=labelSelectInput)

txtInputFile = tk.Text(root, height = 1, width = 30)
txtInputFile.config(font=('helvetica', 16))
txtInputFile.pack()
canvas.create_window(350, 150, window=txtInputFile)

labelNotification = tk.Label(root, text="Notification:",  fg="black" , bg=bgcolor)
labelNotification.config(font=('helvetica', 16))
canvas.create_window(100, 250, window=labelNotification)

txtNotification = tk.Text(root, height = 3, width = 40)
txtNotification.config(font=('helvetica', 16))
txtNotification.pack()
canvas.create_window(450, 250, window=txtNotification)


5

In [3]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result


In [4]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions=['calm', 'happy', 'fearful', 'disgust']


In [5]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("C://Users//DELL//Desktop//INPUT//RAVDESS//Actor_*//*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [6]:
def visualize(path: str):
    raw = wave.open(path)
    signal = raw.readframes(-1)
    signal = np.frombuffer(signal, dtype ="int16")
    f_rate = raw.getframerate()
    time = np.linspace(
        0, # start
        len(signal) / f_rate,
        num = len(signal)
    )
    plt.figure(1)
    plt.title("Sound Wave")
    plt.xlabel("Time")
    plt.plot(time, signal)
    plt.show()

In [7]:
def SelectInputFile():
    global filename
    file = askopenfile(mode ='r', filetypes =[('All Files', '*.wav')]) 
    if file is not None: 
        txtInputFile.delete("1.0",tk.END)
        filepath = file.name
        dirname, basename = os.path.split(filepath)
        txtInputFile.insert(tk.END,basename)
        filename = file

In [8]:
def PlayAudio():
    global filename
    if filename is not None: 
        playsound(filename.name)
        visualize(filename.name)
        

In [9]:
def TrainModel():
    global model
    x_train,x_test,y_train,y_test=load_data(test_size=0.25)
    #print((x_train.shape[0], x_test.shape[0]))
    #print(f'Features extracted: {x_train.shape[1]}')
    model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
    model.fit(x_train,y_train)
    txtNotification.delete("1.0",tk.END)
    txtNotification.insert(tk.END,'Training Done Successfully' )

In [10]:
def Predict():
    global model
    if filename is not None:
        playsound(filename.name)
        feature=extract_feature(filename.name, mfcc=True, chroma=True, mel=True)
        feature = feature.reshape(1, -1)
        y_pred=model.predict(feature)
        #print("y_pred:",y_pred)
        txtNotification.delete("1.0",tk.END)
        txtNotification.insert(tk.END,y_pred[0] )

In [11]:
def Quit():
    root.destroy()
    sys.exit()

In [ ]:
btnBrowse = tk.Button(text='Browse', command=SelectInputFile, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(600, 150, window=btnBrowse)

btnX = 200
btnY = 400

btnPlayAudio = tk.Button(text='Play Audio', command=PlayAudio, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(btnX, btnY, window=btnPlayAudio)

btnX = btnX + 150
btnTrainModel = tk.Button(text='Train Model', command=TrainModel, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(btnX, btnY, window=btnTrainModel)

btnX = btnX + 150
btnPredict = tk.Button(text='Predict', command=Predict, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(btnX, btnY, window=btnPredict)

btnX = btnX + 100
btnQuit = tk.Button(text='Quit', command=Quit, bg='green', fg='white', font=('helvetica', 12, 'bold'))
canvas.create_window(btnX, btnY, window=btnQuit)

root.mainloop()

C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[0. 0. 0. ... 0. 0. 0.] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[0.0000000e+00 3.0517578e-05 3.0517578e-05 ... 0.0000000e+00 0.0000000e+00
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00 -3.0517578e-05 ... -3.0517578e-05
 -3.0517578e-05 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users

C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.0517578e-05
 -3.0517578e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 9.1552734e-05 1.2207031e-04
 1.2207031e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.8310547e-04 6.1035156e-05
 0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(l

C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[ 0.0007019   0.0012207   0.00125122 ...  0.          0.00039673
 -0.00048828] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 6.1035156e-05 9.1552734e-05
 6.1035156e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -6.1035156e-05
 -3.0517578e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.featu

C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[-3.0517578e-05 -3.0517578e-05 -3.0517578e-05 ...  0.0000000e+00
 -3.0517578e-05  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[-3.0517578e-05 -3.0517578e-05  0.0000000e+00 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[-3.0517578e-05 -6.1035156e-05 -3.0517578e-05 ...  9.1552734e-05
  1.2207031e-04  9.1552734e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  m

C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[-6.1035156e-05 -3.0517578e-05 -6.1035156e-05 ... -3.3569336e-04
 -3.9672852e-04 -4.2724609e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[-3.0517578e-05  4.2724609e-04  6.4086914e-04 ...  1.2207031e-04
  9.1552734e-05  1.5258789e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[-6.1035156e-04 -5.7983398e-04 -5.7983398e-04 ... -3.0517578e-05
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  m

...

C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[ 1.8310547e-04  1.8310547e-04  6.1035156e-05 ... -3.0517578e-05
  0.0000000e+00 -3.0517578e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[0.00027466 0.00027466 0.00027466 ... 0.00018311 0.00018311 0.00012207] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[-9.1552734e-05 -9.1552734e-05 -1.5258789e-04 ...  1.2207031e-04
  9.1552734e-05  1.2207031e-04] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.featur

C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[-3.0517578e-05 -3.0517578e-05 -3.0517578e-05 ...  0.0000000e+00
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[ 0.00036621  0.00045776  0.00048828 ... -0.00021362 -0.00021362
 -0.00021362] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[ 0.00067139  0.00076294  0.00106812 ...  0.00344849 -0.00195312
 -0.00204468] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectr

C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[ 0.          0.          0.         ... -0.00057983 -0.00054932
 -0.00057983] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[ 0.0000000e+00 -3.0517578e-05  0.0000000e+00 ...  3.0517578e-05
  0.0000000e+00  0.0000000e+00] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3987746674.py:15: FutureWarning: Pass y=[0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 6.1035156e-05 9.1552734e-05
 9.1552734e-05] as keyword args. From version 0.10 passing these as positional arguments will result in an error
  mel=np.mean(librosa.featu


    Error 263 for command:
        close C:/Users/DELL/Desktop/untitlecyyd.wav
    The specified device is not open or is not recognized by MCI.
Failed to close the file: C:/Users/DELL/Desktop/untitlecyyd.wav
Exception in Tkinter callback
Traceback (most recent call last):
  File "A:\anaconda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3889356443.py", line 4, in Predict
    playsound(filename.name)
  File "A:\anaconda\Lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "A:\anaconda\Lib\site-packages\playsound.py", line 64, in winCommand
    raise PlaysoundException(exceptionMessage)
playsound.PlaysoundException: 
    Error 296 for command:
        open C:/Users/DELL/Desktop/untitlecyyd.wav
    The specified file cannot be played on the specified MCI device.  The file may be corrupt, not in the correct format, or no


    Error 296 for command:
        open C:/Users/DELL/Desktop/03-01-03-02-01-02-nnn09.wav
    The specified file cannot be played on the specified MCI device.  The file may be corrupt, not in the correct format, or no file handler available for this format.

    Error 263 for command:
        close C:/Users/DELL/Desktop/03-01-03-02-01-02-nnn09.wav
    The specified device is not open or is not recognized by MCI.
Failed to close the file: C:/Users/DELL/Desktop/03-01-03-02-01-02-nnn09.wav
Exception in Tkinter callback
Traceback (most recent call last):
  File "A:\anaconda\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_12108\3889356443.py", line 4, in Predict
    playsound(filename.name)
  File "A:\anaconda\Lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "A:\anaconda\Lib\site-packages\playsound.py", line 64, in winCommand
